In [1]:
cd ..

/Users/shelvia.hotama/IdeaProjects/airbnb-ml


In [15]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import src.transform as trans

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [16]:
from datetime import date, timedelta

today = date.today() - timedelta(1)
currentDate = today.strftime("%Y-%m-%d")

In [17]:
import s3fs
import pyarrow.parquet as pq

s3 = s3fs.S3FileSystem()

filePath = 's3://airbnb-barcelona/valid/currentDate=%s' % currentDate
airbnb_df = pq.ParquetDataset(filePath, filesystem=s3).read_pandas().to_pandas()

airbnb = airbnb_df.drop(columns=[
    'rowId',
    'id',
    'host_location',
    'host_neighbourhood',
    'street',
    'neighbourhood',
    'neighbourhood_cleansed',
    'market',
    'license',
    'zipcode'
])

print(airbnb_df.shape)
print(airbnb_df.dtypes.value_counts())

(20428, 66)
float64    36
object     30
dtype: int64


In [24]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from collections import Counter

exploded_amenities_df = airbnb_df['amenities'].apply(trans.explode_amenities_string)

mlb = MultiLabelBinarizer()
one_hot_encoded_amenities = pd.DataFrame(mlb.fit_transform(exploded_amenities_df),
                   columns=mlb.classes_,
                   index=exploded_amenities_df.index)

In [ ]:

amenities_dict = dict(Counter(trans.flatten(exploded_amenities_df.values)))
keys_to_remove = []

for amenity, count in amenities_dict.items():
    if count < 51:
        keys_to_remove.append(amenity)
        

In [ ]:
irrelevant_amenities = ['1125', '1', '', 'translation missing: en.hosting_amenity_49', 'translation missing: en.hosting_amenity_50']
keys_to_remove += irrelevant_amenities
amenities_df = one_hot_encoded_amenities.drop(keys_to_remove, axis=1)
amenities_df.head(5)